In [28]:
import os
import random
import numpy as np
import pandas as pd
import pickle
import warnings
import math
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

# Set deterministic behavior for CUDA (set before torch imports)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

import optuna

# Deterministic seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    return seed

# Global device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device} with {torch.cuda.device_count()} GPU(s)")

Using device: cuda with 5 GPU(s)


### Import Data

In [29]:
# Adjust the data directory as needed
DL_DIR = "../../data/deep_learning"
# Load the regression split dictionary.
with open(f'{DL_DIR}/comb_reg_dict.pkl', 'rb') as f:
    comb_reg_dict = pickle.load(f)

with open(f'{DL_DIR}/fitbit_reg_dict.pkl', 'rb') as f:
    fitbit_reg_dict = pickle.load(f)

# Load the classification split dictionary.
with open(f'{DL_DIR}/comb_class_dict.pkl', 'rb') as f:
    comb_class_dict = pickle.load(f)

with open(f'{DL_DIR}/fitbit_class_dict.pkl', 'rb') as f:
    fitbit_class_dict = pickle.load(f) 

### Utility: compute output length after conv and pooling

In [30]:
def conv_output_length(L_in, kernel_size, stride, padding, dilation):
    return (L_in + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1

def pool_output_length(L_in, pool_kernel):
    # Assume stride==kernel size for pooling
    return L_in // pool_kernel

# Build subject-level dataset (each row is one subject)
def create_subject_dataset(df, outcome_col="SI_mean", use_weights=True):
    """
    Aggregates records for each subject into a subject-level sample.
    If use_weights is False, then sample_weight is set to 1.0 for all subjects.
    """
    exclude_cols = ["PatientID", "timepoints", "si_kde_weight", "SI_mean", "is_SI", "SI_level"]
    predictor_cols = [col for col in df.columns if col not in (exclude_cols + [outcome_col])]
    
    subject_data = []
    for pid, group in df.groupby("PatientID"):
        group_sorted = group.sort_values("timepoints")
        X = group_sorted[predictor_cols].values.T  # shape: (n_features, 39)
        y = group_sorted[outcome_col].iloc[0]
        if use_weights and "si_kde_weight" in group_sorted.columns:
            weight = group_sorted["si_kde_weight"].iloc[0]
        else:
            weight = 1.0
        record = {"PatientID": pid, "X": X, outcome_col: y, "sample_weight": weight}
        if outcome_col == "is_SI" and "SI_mean" in group_sorted.columns:
            record["SI_mean"] = group_sorted["SI_mean"].iloc[0]
        subject_data.append(record)
    subj_df = pd.DataFrame(subject_data)
    subj_df[f"{outcome_col}_bin"] = np.round(subj_df[outcome_col]).astype(int)
    return subj_df, predictor_cols


def save_results_pickle(result_dict, model_name, use_sample_weights=True):
    """
    Saves the result pickle to a folder "search". If use_sample_weights is False,
    appends '_nw' to the filename.
    """
    save_folder = "search"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    if use_sample_weights:
        filename = os.path.join(save_folder, f"{model_name}_deep_search_results.pkl")
    else:
        filename = os.path.join(save_folder, f"{model_name}_nw_deep_search_results.pkl")
    with open(filename, "wb") as f:
        pickle.dump(result_dict, f)
    print(f"Results saved to {filename}")
    
def get_stratified_cv_splits(df, subject_id="PatientID", target_var="SI_mean", n_splits=5):
    """
    Performs stratified K-fold cross validation at the subject level.
    
    Parameters:
      df : pandas.DataFrame
          The original dataframe containing repeated measures.
      subject_id : str
          The column name for the subject ID (e.g., "PatientID").
      target_var : str
          The target variable; for regression use "SI_mean" and for classification use "is_SI".
      n_splits : int
          Number of folds for cross validation.
    
    Returns:
      splits : list of tuples
          A list where each element is a tuple (train_df, test_df) corresponding
          to one fold. Each dataframe contains all rows (i.e. repeated measures) for the patients in that fold.
    
    Behavior:
      - Isolates unique patient IDs and their target variable by dropping duplicates.
      - If target_var is "SI_mean", creates a new column "SI_mean_levels" (rounded SI_mean).
      - Uses the resulting column as the stratification column.
      - Performs stratified K-fold CV and then subsets the original dataframe based on the patient IDs.
    """
    # Create a subject-level dataframe (unique patient IDs with their target variable)
    subject_df = df[[subject_id, target_var]].drop_duplicates(subset=[subject_id]).copy()
    
    # For regression: create a new column with the rounded SI_mean values.
    if target_var == "SI_mean":
        subject_df["SI_mean_levels"] = subject_df[target_var].round().astype(int)
        strat_col = "SI_mean_levels"
    else:
        strat_col = target_var  # For classification, use the target directly.
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    splits = []
    
    # Get the subject IDs and stratification labels
    subjects = subject_df[subject_id].values
    strat_labels = subject_df[strat_col].values
    
    # For each fold, retrieve patient IDs and then subset the original dataframe.
    for train_idx, test_idx in skf.split(subjects, strat_labels):
        train_patient_ids = subject_df.iloc[train_idx][subject_id].values
        test_patient_ids  = subject_df.iloc[test_idx][subject_id].values
        train_split = df[df[subject_id].isin(train_patient_ids)]
        test_split  = df[df[subject_id].isin(test_patient_ids)]
        splits.append((train_split, test_split))
    
    return splits




###  Deep Search Objective Functions for regression

Comb Deep Search

In [31]:
# Non-weighted version (use_weights=False)
def objective_regression_comb_nw(trial, data_dict):
    """
    Non-weighted CNN regression for comb_reg.
    Tuned hyperparameters are the same as in the weighted version.
    """
    set_seed(42)
    splits = get_stratified_cv_splits(
        data_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5
    )
    overall_rmse_list = []
    bin_rmse_dict = {str(b): [] for b in range(1, 6)}
    
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.5)
    n_filters_0 = trial.suggest_int("n_filters_0", 8, 64, step=8)
    kernel_size_0 = trial.suggest_int("kernel_size_0", 3, 7, step=2)
    n_conv = trial.suggest_int("n_conv", 1, 3)
    batch_size = 32

    for train_split, val_split in splits:
        # Force non-weighted subject-level dataset
        train_df, _ = create_subject_dataset(train_split, outcome_col="SI_mean", use_weights=False)
        val_df, _   = create_subject_dataset(val_split, outcome_col="SI_mean", use_weights=False)
        
        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["SI_mean"].values
        # sample_weight will be 1.0 for every subject
        w_train = train_df["sample_weight"].values  
        X_val   = np.stack(val_df["X"].values, axis=0)
        y_val   = val_df["SI_mean"].values
        val_bins = val_df["SI_mean_bin"].values
        
        n_subjects, input_channels, seq_len = X_train.shape
        layers = []
        current_channels = input_channels
        current_seq_len = seq_len
        for i in range(n_conv):
            padding = (kernel_size_0 - 1) // 2
            conv = nn.Conv1d(in_channels=current_channels,
                             out_channels=n_filters_0,
                             kernel_size=kernel_size_0,
                             stride=1,
                             padding=padding)
            layers.extend([conv, nn.ReLU(), nn.Dropout(dropout_prob)])
            current_channels = n_filters_0
            current_seq_len = conv_output_length(current_seq_len, kernel_size_0, 1, padding, 1)
        conv_net = nn.Sequential(*layers)
        flattened_dim = current_channels * current_seq_len
        fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))
        
        class CombCNN(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(CombCNN, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
                
        model = CombCNN(conv_net, fc_net).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
            
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.MSELoss(reduction="none")
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(w_train, dtype=torch.float32)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, weight_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                weight_batch = weight_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = (loss_fn(outputs, y_batch).view(-1) * weight_batch).mean()
                loss.backward()
                optimizer.step()
                
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy()
        fold_rmse = np.sqrt(np.mean((preds - y_val.reshape(-1, 1))**2))
        overall_rmse_list.append(fold_rmse)
        for b in range(1, 6):
            idx = np.where(val_bins == b)[0]
            if len(idx) > 0:
                bin_rmse = np.sqrt(np.mean((preds[idx] - y_val[idx].reshape(-1, 1))**2))
                bin_rmse_dict[str(b)].append(bin_rmse)
                
    overall_mean_rmse = np.mean(overall_rmse_list)
    overall_std_rmse = np.std(overall_rmse_list)
    bin_mean_rmse = {str(b): np.mean(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    bin_std_rmse  = {str(b): np.std(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    
    trial.set_user_attr("overall_mean_rmse", overall_mean_rmse)
    trial.set_user_attr("overall_std_rmse", overall_std_rmse)
    trial.set_user_attr("bin_mean_rmse", bin_mean_rmse)
    trial.set_user_attr("bin_std_rmse", bin_std_rmse)
    
    return overall_mean_rmse


def objective_regression_comb(trial, data_dict):
    """
    Objective function for comb_reg.
    Tunable hyperparameters:
      - lr
      - dropout_prob
      - num_epochs
      - n_filters_0
      - n_cov: number of convolutional layers (1 to 3)
      - use_regularization: whether to add weight decay
      - dilation_0: dilation for all conv layers
    """
    set_seed(42)
    # Use the provided stratified CV splits (using repeated measures)
    splits = get_stratified_cv_splits(data_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)

    overall_rmse_list = []
    bin_rmse_dict = {str(b): [] for b in range(1, 6)}
    
    # Tunable hyperparameters
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.5)
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    n_filters_0 = trial.suggest_int("n_filters_0", 8, 64, step=8)
    n_cov = trial.suggest_int("n_cov", 1, 3)  # number of convolutional layers
    use_regularization = trial.suggest_categorical("use_regularization", [True, False])
    dilation_0 = trial.suggest_int("dilation_0", 1, 3)
    batch_size = 32  # fixed

    for train_split, val_split in splits:
        # Build subject-level datasets for training and validation
        train_df, _ = create_subject_dataset(train_split, outcome_col="SI_mean")
        val_df, _ = create_subject_dataset(val_split, outcome_col="SI_mean")
        
        X_train = np.stack(train_df["X"].values, axis=0)  # shape: (n_subjects, n_features, seq_len)
        y_train = train_df["SI_mean"].values
        w_train = train_df["sample_weight"].values
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["SI_mean"].values
        val_bins = val_df["SI_mean_bin"].values
        
        n_subjects, input_channels, seq_len = X_train.shape

        # Build a conv net with n_cov layers.
        layers = []
        current_channels = input_channels
        current_seq_len = seq_len
        for i in range(n_cov):
            # For kernel size 3, set padding to preserve dimension:
            padding = dilation_0 * ((3 - 1) // 2)
            conv = nn.Conv1d(in_channels=current_channels,
                             out_channels=n_filters_0,
                             kernel_size=3,
                             stride=1,
                             padding=padding,
                             dilation=dilation_0)
            layers.append(conv)
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_prob))
            current_channels = n_filters_0
            current_seq_len = conv_output_length(current_seq_len, kernel_size=3, stride=1, padding=padding, dilation=dilation_0)
        conv_net = nn.Sequential(*layers)
        flattened_dim = current_channels * current_seq_len
        fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))

        class CombCNN(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(CombCNN, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)

        model = CombCNN(conv_net, fc_net).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)

        # If using regularization, add a small weight decay.
        weight_decay = 1e-4 if use_regularization else 0.0
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        loss_fn = nn.MSELoss(reduction="none")

        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(w_train, dtype=torch.float32)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, weight_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                weight_batch = weight_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = (loss_fn(outputs, y_batch).view(-1) * weight_batch).mean()
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy()
        fold_rmse = np.sqrt(np.mean((preds - y_val.reshape(-1, 1)) ** 2))
        overall_rmse_list.append(fold_rmse)

        # Compute per-bin RMSE.
        for b in range(1, 6):
            idx = np.where(val_bins == b)[0]
            if len(idx) > 0:
                bin_rmse = np.sqrt(np.mean((preds[idx] - y_val[idx].reshape(-1, 1)) ** 2))
                bin_rmse_dict[str(b)].append(bin_rmse)

    overall_mean_rmse = np.mean(overall_rmse_list)
    overall_std_rmse = np.std(overall_rmse_list)
    bin_mean_rmse = {str(b): np.mean(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    bin_std_rmse = {str(b): np.std(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                    for b in range(1, 6)}

    trial.set_user_attr("overall_mean_rmse", overall_mean_rmse)
    trial.set_user_attr("overall_std_rmse", overall_std_rmse)
    trial.set_user_attr("bin_mean_rmse", bin_mean_rmse)
    trial.set_user_attr("bin_std_rmse", bin_std_rmse)

    return overall_mean_rmse


def run_deep_search_comb_reg(data_dict, use_sample_weights, model_name, n_trials=75):
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    def print_progress(study, trial):
        print(f"{model_name} trial {len(study.trials)}/{n_trials}\r", end="", flush=True)
    study = optuna.create_study(direction="minimize")
    optimize_fn = objective_regression_comb if use_sample_weights else objective_regression_comb_nw
    study.optimize(lambda trial: optimize_fn(trial, data_dict),
                   n_trials=n_trials, callbacks=[print_progress])

    mean_rows = []
    std_rows = []
    for t in study.trials:
        row_mean = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "1": t.user_attrs["bin_mean_rmse"].get("1", float('nan')),
            "2": t.user_attrs["bin_mean_rmse"].get("2", float('nan')),
            "3": t.user_attrs["bin_mean_rmse"].get("3", float('nan')),
            "4": t.user_attrs["bin_mean_rmse"].get("4", float('nan')),
            "5": t.user_attrs["bin_mean_rmse"].get("5", float('nan')),
            "overall": t.user_attrs["overall_mean_rmse"],
            "config": t.params
        }
        row_std = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "1": t.user_attrs["bin_std_rmse"].get("1", float('nan')),
            "2": t.user_attrs["bin_std_rmse"].get("2", float('nan')),
            "3": t.user_attrs["bin_std_rmse"].get("3", float('nan')),
            "4": t.user_attrs["bin_std_rmse"].get("4", float('nan')),
            "5": t.user_attrs["bin_std_rmse"].get("5", float('nan')),
            "overall": t.user_attrs["overall_std_rmse"],
            "config": t.params
        }
        mean_rows.append(row_mean)
        std_rows.append(row_std)

    columns = ["model", "type", "1", "2", "3", "4", "5", "overall", "config"]
    mean_df = pd.DataFrame(mean_rows, columns=columns)
    std_df = pd.DataFrame(std_rows, columns=columns)

    best_trial = study.best_trial
    optimal_configuration = {
        "value": best_trial.value,
        "params": best_trial.params,
        "user_attrs": best_trial.user_attrs
    }

    result_dict = {
        "mean_rmse": mean_df,
        "std_rmse": std_df,
        "optimal_configuration": optimal_configuration
    }
    return result_dict

Fitbit Deep Search

In [32]:
# Non-weighted version
def objective_regression_fitbit_nw(trial, data_dict):
    """
    Non-weighted CNN regression for fitbit_reg.
    Tuned hyperparameters:
      - dropout_prob
      - n_filters_0
      - num_epochs
    """
    set_seed(42)
    splits = get_stratified_cv_splits(
        data_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5
    )
    overall_rmse_list = []
    bin_rmse_dict = {str(b): [] for b in range(1, 6)}
    
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.5)
    n_filters_0 = trial.suggest_int("n_filters_0", 8, 64, step=8)
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    
    kernel_size = 3
    stride = 1
    padding = (kernel_size - 1) // 2
    batch_size = 32
    
    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="SI_mean", use_weights=False)
        val_df, _ = create_subject_dataset(val_split, outcome_col="SI_mean", use_weights=False)
        
        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["SI_mean"].values
        w_train = train_df["sample_weight"].values  # should be 1.0
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["SI_mean"].values
        val_bins = val_df["SI_mean_bin"].values
        
        n_subjects, input_channels, seq_len = X_train.shape
        conv = nn.Conv1d(in_channels=input_channels,
                         out_channels=n_filters_0,
                         kernel_size=kernel_size,
                         stride=stride,
                         padding=padding)
        layers = [conv, nn.ReLU(), nn.Dropout(dropout_prob)]
        conv_net = nn.Sequential(*layers)
        new_seq_len = conv_output_length(seq_len, kernel_size, stride, padding, dilation=1)
        flattened_dim = n_filters_0 * new_seq_len
        fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))
        
        class FitRegCNN(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(FitRegCNN, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
                
        model = FitRegCNN(conv_net, fc_net).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
            
        lr_fixed = 1e-3
        optimizer = optim.Adam(model.parameters(), lr=lr_fixed)
        loss_fn = nn.MSELoss(reduction="none")
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(w_train, dtype=torch.float32)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, weight_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                weight_batch = weight_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = (loss_fn(outputs, y_batch).view(-1) * weight_batch).mean()
                loss.backward()
                optimizer.step()
                
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy()
        fold_rmse = np.sqrt(np.mean((preds - y_val.reshape(-1, 1))**2))
        overall_rmse_list.append(fold_rmse)
        for b in range(1, 6):
            idx = np.where(val_bins == b)[0]
            if len(idx) > 0:
                bin_rmse = np.sqrt(np.mean((preds[idx] - y_val[idx].reshape(-1, 1))**2))
                bin_rmse_dict[str(b)].append(bin_rmse)
                
    overall_mean_rmse = np.mean(overall_rmse_list)
    overall_std_rmse = np.std(overall_rmse_list)
    bin_mean_rmse = {str(b): np.mean(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    bin_std_rmse  = {str(b): np.std(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    
    trial.set_user_attr("overall_mean_rmse", overall_mean_rmse)
    trial.set_user_attr("overall_std_rmse", overall_std_rmse)
    trial.set_user_attr("bin_mean_rmse", bin_mean_rmse)
    trial.set_user_attr("bin_std_rmse", bin_std_rmse)
    
    return overall_mean_rmse

def objective_regression_fitbit(trial, data_dict):
    """
    Objective function for fitbit_reg.
    Tunable hyperparameters:
      - drop_out_prb (dropout probability)
      - lr
      - use_regularization
      - num_epochs
      - n_filters_0
    """
    set_seed(42)
    splits = get_stratified_cv_splits(data_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)

    overall_rmse_list = []
    bin_rmse_dict = {str(b): [] for b in range(1, 6)}

    dropout_prob = trial.suggest_float("drop_out_prb", 0.1, 0.5)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    use_regularization = trial.suggest_categorical("use_regularization", [True, False])
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    n_filters_0 = trial.suggest_int("n_filters_0", 8, 64, step=8)

    # Fixed parameters for fitbit_reg:
    batch_size = 32
    kernel_size = 3
    stride = 1
    dilation = 1
    padding = (kernel_size - 1) // 2

    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="SI_mean")
        val_df, _ = create_subject_dataset(val_split, outcome_col="SI_mean")

        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["SI_mean"].values
        w_train = train_df["sample_weight"].values

        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["SI_mean"].values
        val_bins = val_df["SI_mean_bin"].values

        n_subjects, input_channels, seq_len = X_train.shape

        layers = []
        conv0 = nn.Conv1d(in_channels=input_channels,
                          out_channels=n_filters_0,
                          kernel_size=kernel_size,
                          stride=stride,
                          dilation=dilation,
                          padding=padding)
        layers.append(conv0)
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_prob))
        conv_net = nn.Sequential(*layers)
        new_seq_len = conv_output_length(seq_len, kernel_size, stride, padding, dilation)
        flattened_dim = n_filters_0 * new_seq_len
        fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))

        class FitRegCNN(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(FitRegCNN, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)

        model = FitRegCNN(conv_net, fc_net).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)

        weight_decay = 1e-4 if use_regularization else 0.0
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        loss_fn = nn.MSELoss(reduction="none")

        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(w_train, dtype=torch.float32)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, weight_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                weight_batch = weight_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = (loss_fn(outputs, y_batch).view(-1) * weight_batch).mean()
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy()
        fold_rmse = np.sqrt(np.mean((preds - y_val.reshape(-1, 1)) ** 2))
        overall_rmse_list.append(fold_rmse)

        for b in range(1, 6):
            idx = np.where(val_bins == b)[0]
            if len(idx) > 0:
                bin_rmse = np.sqrt(np.mean((preds[idx] - y_val[idx].reshape(-1, 1)) ** 2))
                bin_rmse_dict[str(b)].append(bin_rmse)

    overall_mean_rmse = np.mean(overall_rmse_list)
    overall_std_rmse = np.std(overall_rmse_list)
    bin_mean_rmse = {str(b): np.mean(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    bin_std_rmse = {str(b): np.std(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                    for b in range(1, 6)}

    trial.set_user_attr("overall_mean_rmse", overall_mean_rmse)
    trial.set_user_attr("overall_std_rmse", overall_std_rmse)
    trial.set_user_attr("bin_mean_rmse", bin_mean_rmse)
    trial.set_user_attr("bin_std_rmse", bin_std_rmse)

    return overall_mean_rmse


def run_deep_search_fitbit_reg(data_dict, use_sample_weights, model_name, n_trials=75):
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    def print_progress(study, trial):
        print(f"{model_name} trial {len(study.trials)}/{n_trials}\r", end="", flush=True)
    study = optuna.create_study(direction="minimize")
    optimize_fn = objective_regression_fitbit if use_sample_weights else objective_regression_fitbit_nw
    study.optimize(lambda trial: optimize_fn(trial, data_dict),
                   n_trials=n_trials, callbacks=[print_progress])

    mean_rows = []
    std_rows = []
    for t in study.trials:
        row_mean = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "1": t.user_attrs["bin_mean_rmse"].get("1", float('nan')),
            "2": t.user_attrs["bin_mean_rmse"].get("2", float('nan')),
            "3": t.user_attrs["bin_mean_rmse"].get("3", float('nan')),
            "4": t.user_attrs["bin_mean_rmse"].get("4", float('nan')),
            "5": t.user_attrs["bin_mean_rmse"].get("5", float('nan')),
            "overall": t.user_attrs["overall_mean_rmse"],
            "config": t.params
        }
        row_std = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "1": t.user_attrs["bin_std_rmse"].get("1", float('nan')),
            "2": t.user_attrs["bin_std_rmse"].get("2", float('nan')),
            "3": t.user_attrs["bin_std_rmse"].get("3", float('nan')),
            "4": t.user_attrs["bin_std_rmse"].get("4", float('nan')),
            "5": t.user_attrs["bin_std_rmse"].get("5", float('nan')),
            "overall": t.user_attrs["overall_std_rmse"],
            "config": t.params
        }
        mean_rows.append(row_mean)
        std_rows.append(row_std)

    columns = ["model", "type", "1", "2", "3", "4", "5", "overall", "config"]
    mean_df = pd.DataFrame(mean_rows, columns=columns)
    std_df = pd.DataFrame(std_rows, columns=columns)

    best_trial = study.best_trial
    optimal_configuration = {"value": best_trial.value,
                             "params": best_trial.params,
                             "user_attrs": best_trial.user_attrs}

    result_dict = {"mean_rmse": mean_df,
                   "std_rmse": std_df,
                   "optimal_configuration": optimal_configuration}
    return result_dict

### Run deep search for regression

#### Weighted

In [12]:
# Run deep search for COMB dataset (n_trials determines how many steps/trials are executed)
results_comb_reg = run_deep_search_comb_reg(comb_reg_dict, use_sample_weights=True, model_name="comb_reg_deep", n_trials=75)
save_results_pickle(results_comb_reg, "comb_reg_deep")
results_comb_reg["mean_rmse"]


Results saved to search/comb_reg_deep_deep_search_results.pkl


,model,type,1,2,3,4,5,overall,config
0,comb_reg_deep,weighted,1.393811,0.870844,0.869801,1.536806,2.297815,1.332119,"{'lr': 0.0005288622240373623, 'dropout_prob': ..."
1,comb_reg_deep,weighted,1.085239,0.628376,1.017290,1.863034,2.829652,1.064307,"{'lr': 4.896278079138998e-05, 'dropout_prob': ..."
2,comb_reg_deep,weighted,1.709774,1.043727,0.540656,1.181194,2.062841,1.606519,"{'lr': 8.546548868916738e-05, 'dropout_prob': ..."
3,comb_reg_deep,weighted,1.425002,1.252953,1.516907,2.155115,3.191589,1.468563,"{'lr': 0.007056872698036484, 'dropout_prob': 0..."
4,comb_reg_deep,weighted,1.598624,1.046183,0.746520,1.453205,2.050083,1.520920,"{'lr': 0.0002553399922608638, 'dropout_prob': ..."
...,...,...,...,...,...,...,...,...,...
70,comb_reg_deep,weighted,0.830737,0.513131,1.227953,2.022883,2.861890,0.860659,"{'lr': 1.7223038444033673e-05, 'dropout_prob':..."
71,comb_reg_deep,weighted,0.248935,0.886722,1.920040,2.771035,3.790356,0.661265,"{'lr': 1.0093409334112497e-05, 'dropout_prob':..."
72,comb_reg_deep,weighted,0.408769,0.679306,1.666788,2.502117,3.518366,0.653402,"{'lr': 1.2190224984562657e-05, 'dropout_prob':..."
73,comb_reg_deep,weighted,1.540365,0.876328,0.541653,1.361522,2.039991,1.446638,"{'lr': 2.3097356466409334e-05, 'dropout_prob':..."


In [13]:
# Run deep search for FITBIT dataset
results_fitbit_reg = run_deep_search_fitbit_reg(fitbit_reg_dict, use_sample_weights=True, model_name="fitbit_reg_deep", n_trials=75)
save_results_pickle(results_fitbit_reg, "fitbit_reg_deep")
results_fitbit_reg["mean_rmse"]



Results saved to search/fitbit_reg_deep_deep_search_results.pkl


,model,type,1,2,3,4,5,overall,config
0,comb_reg_deep,weighted,1.393811,0.870844,0.869801,1.536806,2.297815,1.332119,"{'lr': 0.0005288622240373623, 'dropout_prob': ..."
1,comb_reg_deep,weighted,1.085239,0.628376,1.017290,1.863034,2.829652,1.064307,"{'lr': 4.896278079138998e-05, 'dropout_prob': ..."
2,comb_reg_deep,weighted,1.709774,1.043727,0.540656,1.181194,2.062841,1.606519,"{'lr': 8.546548868916738e-05, 'dropout_prob': ..."
3,comb_reg_deep,weighted,1.425002,1.252953,1.516907,2.155115,3.191589,1.468563,"{'lr': 0.007056872698036484, 'dropout_prob': 0..."
4,comb_reg_deep,weighted,1.598624,1.046183,0.746520,1.453205,2.050083,1.520920,"{'lr': 0.0002553399922608638, 'dropout_prob': ..."
...,...,...,...,...,...,...,...,...,...
70,comb_reg_deep,weighted,0.830737,0.513131,1.227953,2.022883,2.861890,0.860659,"{'lr': 1.7223038444033673e-05, 'dropout_prob':..."
71,comb_reg_deep,weighted,0.248935,0.886722,1.920040,2.771035,3.790356,0.661265,"{'lr': 1.0093409334112497e-05, 'dropout_prob':..."
72,comb_reg_deep,weighted,0.408769,0.679306,1.666788,2.502117,3.518366,0.653402,"{'lr': 1.2190224984562657e-05, 'dropout_prob':..."
73,comb_reg_deep,weighted,1.540365,0.876328,0.541653,1.361522,2.039991,1.446638,"{'lr': 2.3097356466409334e-05, 'dropout_prob':..."


### Unweighted

In [39]:
results_comb_reg = run_deep_search_comb_reg(comb_reg_dict, use_sample_weights=False, model_name="comb_reg_deep", n_trials=75)
save_results_pickle(results_comb_reg, "comb_reg_deep_nw")
results_comb_reg["mean_rmse"]

Results saved to search/comb_reg_deep_nw_deep_search_results.pkl


,model,type,1,2,3,4,5,overall,config
0,comb_reg_deep,not weighted,0.219079,0.772274,1.724489,2.688882,3.686512,0.607270,"{'lr': 0.0001445914452794981, 'num_epochs': 7,..."
1,comb_reg_deep,not weighted,0.227203,0.745711,1.702541,2.681855,3.660913,0.601519,"{'lr': 6.0557504510913714e-05, 'num_epochs': 7..."
2,comb_reg_deep,not weighted,0.415848,0.799626,1.659092,2.647804,3.455087,0.682556,"{'lr': 0.007148172666773336, 'num_epochs': 6, ..."
3,comb_reg_deep,not weighted,0.231826,0.765090,1.712052,2.630040,3.649265,0.603878,"{'lr': 0.00012275979082630953, 'num_epochs': 8..."
4,comb_reg_deep,not weighted,0.325820,0.711415,1.633101,2.667351,3.506835,0.625045,"{'lr': 0.0016120799177080705, 'num_epochs': 6,..."
...,...,...,...,...,...,...,...,...,...
70,comb_reg_deep,not weighted,0.222728,0.767614,1.734866,2.676031,3.664791,0.606583,"{'lr': 3.9226215913848744e-05, 'num_epochs': 1..."
71,comb_reg_deep,not weighted,0.262880,0.706867,1.641748,2.626062,3.615995,0.597381,"{'lr': 0.00019525882860361605, 'num_epochs': 1..."
72,comb_reg_deep,not weighted,0.262867,0.704961,1.643646,2.619193,3.614400,0.597078,"{'lr': 0.00015336843667400446, 'num_epochs': 1..."
73,comb_reg_deep,not weighted,0.254887,0.727488,1.676276,2.627156,3.676336,0.602054,"{'lr': 0.00011369846686143195, 'num_epochs': 1..."


In [40]:
results_fitbit_reg = run_deep_search_fitbit_reg(fitbit_reg_dict, use_sample_weights=False, model_name="fitbit_reg_deep", n_trials=75)
save_results_pickle(results_fitbit_reg, "fitbit_reg_deep_nw")
results_fitbit_reg["mean_rmse"]

Results saved to search/fitbit_reg_deep_nw_deep_search_results.pkl


,model,type,1,2,3,4,5,overall,config
0,fitbit_reg_deep,not weighted,0.301444,0.772690,1.660792,2.683249,3.657459,0.630615,"{'dropout_prob': 0.3326082417076779, 'n_filter..."
1,fitbit_reg_deep,not weighted,0.327491,0.711077,1.665755,2.582421,3.602582,0.622867,"{'dropout_prob': 0.3068599720375237, 'n_filter..."
2,fitbit_reg_deep,not weighted,0.292815,0.734227,1.704416,2.666346,3.608171,0.621249,"{'dropout_prob': 0.49178864075773265, 'n_filte..."
3,fitbit_reg_deep,not weighted,0.355127,0.723047,1.658068,2.631583,3.655334,0.641610,"{'dropout_prob': 0.24133697341394322, 'n_filte..."
4,fitbit_reg_deep,not weighted,0.366435,0.742917,1.670036,2.575252,3.600745,0.647544,"{'dropout_prob': 0.12301782875485451, 'n_filte..."
...,...,...,...,...,...,...,...,...,...
70,fitbit_reg_deep,not weighted,0.364246,0.723072,1.632760,2.610837,3.644825,0.644291,"{'dropout_prob': 0.3186437999665475, 'n_filter..."
71,fitbit_reg_deep,not weighted,0.277437,0.764302,1.708684,2.607566,3.617004,0.616972,"{'dropout_prob': 0.4629268393784858, 'n_filter..."
72,fitbit_reg_deep,not weighted,0.276947,0.763884,1.706084,2.609960,3.610045,0.616613,"{'dropout_prob': 0.45369056974403743, 'n_filte..."
73,fitbit_reg_deep,not weighted,0.278165,0.763893,1.707058,2.604729,3.611156,0.616743,"{'dropout_prob': 0.46894321051414145, 'n_filte..."


###  Deep Search Objective Functions for classifcation

### Classification Deep Search

Comb

In [35]:
# Non-weighted version
def objective_classification_comb_nw(trial, data_dict):
    """
    Non-weighted CNN classification for comb_class.
    Tuned hyperparameters:
      - batch_size
      - n_filters_0
    """
    set_seed(42)
    splits = get_stratified_cv_splits(
        data_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5
    )
    overall_acc_list = []
    overall_sens_list = []
    overall_spec_list = []
    
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16)
    n_filters_0 = trial.suggest_int("n_filters_0", 8, 64, step=8)
    
    dropout_prob = 0.2
    lr = 1e-3
    num_epochs = 10
    
    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="is_SI", use_weights=False)
        val_df, _ = create_subject_dataset(val_split, outcome_col="is_SI", use_weights=False)
        
        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["is_SI"].values.astype(np.float32)
        X_val   = np.stack(val_df["X"].values, axis=0)
        y_val   = val_df["is_SI"].values.astype(np.float32)
        
        conv = nn.Conv1d(in_channels=X_train.shape[1],
                         out_channels=n_filters_0,
                         kernel_size=3,
                         stride=1,
                         padding=1)
        layers = [conv, nn.ReLU(), nn.Dropout(dropout_prob)]
        conv_net = nn.Sequential(*layers)
        flattened_dim = n_filters_0 * X_train.shape[2]
        fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))
        
        class CombClassCNN(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(CombClassCNN, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
                
        model = CombClassCNN(conv_net, fc_net).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
            
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.BCEWithLogitsLoss()
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()
                
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            logits = model(X_val_tensor)
            probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
            preds = (probs >= 0.5).astype(np.float32)
        fold_acc = np.mean(preds == y_val)
        TP = np.sum((preds == 1) & (y_val == 1))
        FN = np.sum((preds == 0) & (y_val == 1))
        sensitivity = TP/(TP+FN) if (TP+FN) > 0 else np.nan
        TN = np.sum((preds == 0) & (y_val == 0))
        FP = np.sum((preds == 1) & (y_val == 0))
        specificity = TN/(TN+FP) if (TN+FP) > 0 else np.nan
        
        overall_acc_list.append(fold_acc)
        overall_sens_list.append(sensitivity)
        overall_spec_list.append(specificity)
        
    mean_acc = np.mean(overall_acc_list)
    std_acc = np.std(overall_acc_list)
    mean_sens = np.mean(overall_sens_list)
    std_sens = np.std(overall_sens_list)
    mean_spec = np.mean(overall_spec_list)
    std_spec = np.std(overall_spec_list)
    
    trial.set_user_attr("overall_acc_mean", mean_acc)
    trial.set_user_attr("overall_acc_std", std_acc)
    trial.set_user_attr("sensitivity_mean", mean_sens)
    trial.set_user_attr("sensitivity_std", std_sens)
    trial.set_user_attr("specificity_mean", mean_spec)
    trial.set_user_attr("specificity_std", std_spec)
    
    return 1 - mean_acc

def objective_classification_comb(trial, data_dict):
    """
    Objective function for comb_class.
    Tunable hyperparameters:
      - lr
      - dropout_prob
      - num_epochs
      - batch_size
      - n_filters_0
    """
    set_seed(42)
    splits = get_stratified_cv_splits(data_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)

    overall_acc_list = []
    overall_sens_list = []
    overall_spec_list = []

    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.5)
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16)
    n_filters_0 = trial.suggest_int("n_filters_0", 8, 64, step=8)

    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="is_SI")
        val_df, _ = create_subject_dataset(val_split, outcome_col="is_SI")

        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["is_SI"].values.astype(np.float32)

        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["is_SI"].values.astype(np.float32)

        conv = nn.Conv1d(in_channels=X_train.shape[1],
                         out_channels=n_filters_0,
                         kernel_size=3,
                         stride=1,
                         padding=1)
        layers = [conv, nn.ReLU(), nn.Dropout(dropout_prob)]
        conv_net = nn.Sequential(*layers)
        flattened_dim = n_filters_0 * X_train.shape[2]
        fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))

        class CombClassCNN(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(CombClassCNN, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)

        model = CombClassCNN(conv_net, fc_net).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)

        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.BCEWithLogitsLoss()

        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            logits = model(X_val_tensor)
            probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
            preds = (probs >= 0.5).astype(np.float32)
        fold_acc = np.mean(preds == y_val)
        TP = np.sum((preds == 1) & (y_val == 1))
        FN = np.sum((preds == 0) & (y_val == 1))
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        TN = np.sum((preds == 0) & (y_val == 0))
        FP = np.sum((preds == 1) & (y_val == 0))
        specificity = TN / (TN + FP) if (TN + FP) > 0 else np.nan

        overall_acc_list.append(fold_acc)
        overall_sens_list.append(sensitivity)
        overall_spec_list.append(specificity)

    mean_acc = np.mean(overall_acc_list)
    std_acc = np.std(overall_acc_list)
    mean_sens = np.mean(overall_sens_list)
    std_sens = np.std(overall_sens_list)
    mean_spec = np.mean(overall_spec_list)
    std_spec = np.std(overall_spec_list)

    trial.set_user_attr("overall_acc_mean", mean_acc)
    trial.set_user_attr("overall_acc_std", std_acc)
    trial.set_user_attr("sensitivity_mean", mean_sens)
    trial.set_user_attr("sensitivity_std", std_sens)
    trial.set_user_attr("specificity_mean", mean_spec)
    trial.set_user_attr("specificity_std", std_spec)

    return 1 - mean_acc  # we minimize 1 - accuracy


def run_deep_search_comb_class(data_dict, use_sample_weights, model_name, n_trials=75):
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    def print_progress(study, trial):
        print(f"{model_name} trial {len(study.trials)}/{n_trials}\r", end="", flush=True)
    study = optuna.create_study(direction="minimize")
    optimize_fn = objective_classification_comb if use_sample_weights else objective_classification_comb_nw
    study.optimize(lambda trial: optimize_fn(trial, data_dict),
                   n_trials=n_trials, callbacks=[print_progress])

    mean_rows = []
    std_rows = []
    for t in study.trials:
        row_mean = {
            "model": model_name,
            "type": "weighted",
            "accuracy": t.user_attrs["overall_acc_mean"],
            "sensitivity": t.user_attrs["sensitivity_mean"],
            "specificity": t.user_attrs["specificity_mean"],
            "config": t.params
        }
        row_std = {
            "model": model_name,
            "type": "weighted",
            "accuracy": t.user_attrs["overall_acc_std"],
            "sensitivity": t.user_attrs["sensitivity_std"],
            "specificity": t.user_attrs["specificity_std"],
            "config": t.params
        }
        mean_rows.append(row_mean)
        std_rows.append(row_std)

    columns = ["model", "type", "accuracy", "sensitivity", "specificity", "config"]
    mean_df = pd.DataFrame(mean_rows, columns=columns)
    std_df = pd.DataFrame(std_rows, columns=columns)

    best_trial = study.best_trial
    optimal_configuration = {
        "value": best_trial.value,
        "params": best_trial.params,
        "user_attrs": best_trial.user_attrs
    }

    result_dict = {
        "mean_metrics": mean_df,
        "std_metrics": std_df,
        "optimal_configuration": optimal_configuration
    }
    return result_dict

comb

In [36]:
# Non-weighted version
def objective_classification_fitbit_nw(trial, data_dict):
    """
    Non-weighted CNN classification for fitbit_class.
    Tuned hyperparameters:
      - num_epochs
      - lr
    """
    set_seed(42)
    splits = get_stratified_cv_splits(
        data_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5
    )
    overall_acc_list = []
    overall_sens_list = []
    overall_spec_list = []
    
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    
    n_filters_0 = 32  # fixed default
    dropout_prob = 0.2
    batch_size = 32
    
    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="is_SI", use_weights=False)
        val_df, _ = create_subject_dataset(val_split, outcome_col="is_SI", use_weights=False)
        
        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["is_SI"].values.astype(np.float32)
        X_val   = np.stack(val_df["X"].values, axis=0)
        y_val   = val_df["is_SI"].values.astype(np.float32)
        
        conv = nn.Conv1d(in_channels=X_train.shape[1],
                         out_channels=n_filters_0,
                         kernel_size=3,
                         stride=1,
                         padding=1)
        layers = [conv, nn.ReLU(), nn.Dropout(dropout_prob)]
        conv_net = nn.Sequential(*layers)
        flattened_dim = n_filters_0 * X_train.shape[2]
        fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))
        
        class FitbitClassCNN(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(FitbitClassCNN, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
                
        model = FitbitClassCNN(conv_net, fc_net).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
            
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.BCEWithLogitsLoss()
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()
                
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            logits = model(X_val_tensor)
            probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
            preds = (probs >= 0.5).astype(np.float32)
        fold_acc = np.mean(preds == y_val)
        TP = np.sum((preds == 1) & (y_val == 1))
        FN = np.sum((preds == 0) & (y_val == 1))
        sensitivity = TP/(TP+FN) if (TP+FN) > 0 else np.nan
        TN = np.sum((preds == 0) & (y_val == 0))
        FP = np.sum((preds == 1) & (y_val == 0))
        specificity = TN/(TN+FP) if (TN+FP) > 0 else np.nan
        
        overall_acc_list.append(fold_acc)
        overall_sens_list.append(sensitivity)
        overall_spec_list.append(specificity)
        
    mean_acc = np.mean(overall_acc_list)
    std_acc = np.std(overall_acc_list)
    mean_sens = np.mean(overall_sens_list)
    std_sens = np.std(overall_sens_list)
    mean_spec = np.mean(overall_spec_list)
    std_spec = np.std(overall_spec_list)
    
    trial.set_user_attr("overall_acc_mean", mean_acc)
    trial.set_user_attr("overall_acc_std", std_acc)
    trial.set_user_attr("sensitivity_mean", mean_sens)
    trial.set_user_attr("sensitivity_std", std_sens)
    trial.set_user_attr("specificity_mean", mean_spec)
    trial.set_user_attr("specificity_std", std_spec)
    
    return 1 - mean_acc

def objective_classification_fitbit(trial, data_dict):
    """
    Objective function for fitbit_class.
    Tunable hyperparameters:
      - lr
      - n_filters_0
    Other parameters (dropout, kernel size, etc.) are fixed.
    """
    set_seed(42)
    splits = get_stratified_cv_splits(data_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)

    overall_acc_list = []
    overall_sens_list = []
    overall_spec_list = []

    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    n_filters_0 = trial.suggest_int("n_filters_0", 8, 64, step=8)

    # Fixed parameters:
    dropout_prob = 0.2
    kernel_size = 3
    batch_size = 32

    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="is_SI")
        val_df, _ = create_subject_dataset(val_split, outcome_col="is_SI")

        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["is_SI"].values.astype(np.float32)

        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["is_SI"].values.astype(np.float32)

        conv = nn.Conv1d(in_channels=X_train.shape[1],
                         out_channels=n_filters_0,
                         kernel_size=kernel_size,
                         stride=1,
                         padding=(kernel_size - 1) // 2)
        layers = [conv, nn.ReLU(), nn.Dropout(dropout_prob)]
        conv_net = nn.Sequential(*layers)
        flattened_dim = n_filters_0 * X_train.shape[2]
        fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))

        class FitbitClassCNN(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(FitbitClassCNN, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)

        model = FitbitClassCNN(conv_net, fc_net).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)

        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.BCEWithLogitsLoss()

        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

        model.train()
        for epoch in range(5):  # fixed number of epochs
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            logits = model(X_val_tensor)
            probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
            preds = (probs >= 0.5).astype(np.float32)
        fold_acc = np.mean(preds == y_val)
        TP = np.sum((preds == 1) & (y_val == 1))
        FN = np.sum((preds == 0) & (y_val == 1))
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        TN = np.sum((preds == 0) & (y_val == 0))
        FP = np.sum((preds == 1) & (y_val == 0))
        specificity = TN / (TN + FP) if (TN + FP) > 0 else np.nan

        overall_acc_list.append(fold_acc)
        overall_sens_list.append(sensitivity)
        overall_spec_list.append(specificity)

    mean_acc = np.mean(overall_acc_list)
    std_acc = np.std(overall_acc_list)
    mean_sens = np.mean(overall_sens_list)
    std_sens = np.std(overall_sens_list)
    mean_spec = np.mean(overall_spec_list)
    std_spec = np.std(overall_spec_list)

    trial.set_user_attr("overall_acc_mean", mean_acc)
    trial.set_user_attr("overall_acc_std", std_acc)
    trial.set_user_attr("sensitivity_mean", mean_sens)
    trial.set_user_attr("sensitivity_std", std_sens)
    trial.set_user_attr("specificity_mean", mean_spec)
    trial.set_user_attr("specificity_std", std_spec)

    return 1 - mean_acc


def run_deep_search_fitbit_class(data_dict, use_sample_weights, model_name, n_trials=75):
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    def print_progress(study, trial):
        print(f"{model_name} trial {len(study.trials)}/{n_trials}\r", end="", flush=True)
    study = optuna.create_study(direction="minimize")
    optimize_fn = objective_classification_fitbit if use_sample_weights else objective_classification_fitbit_nw
    study.optimize(lambda trial: optimize_fn(trial, data_dict),
                   n_trials=n_trials, callbacks=[print_progress])

    mean_rows = []
    std_rows = []
    for t in study.trials:
        row_mean = {
            "model": model_name,
            "type": "weighted",
            "accuracy": t.user_attrs["overall_acc_mean"],
            "sensitivity": t.user_attrs["sensitivity_mean"],
            "specificity": t.user_attrs["specificity_mean"],
            "config": t.params
        }
        row_std = {
            "model": model_name,
            "type": "weighted",
            "accuracy": t.user_attrs["overall_acc_std"],
            "sensitivity": t.user_attrs["sensitivity_std"],
            "specificity": t.user_attrs["specificity_std"],
            "config": t.params
        }
        mean_rows.append(row_mean)
        std_rows.append(row_std)

    columns = ["model", "type", "accuracy", "sensitivity", "specificity", "config"]
    mean_df = pd.DataFrame(mean_rows, columns=columns)
    std_df = pd.DataFrame(std_rows, columns=columns)

    best_trial = study.best_trial
    optimal_configuration = {
        "value": best_trial.value,
        "params": best_trial.params,
        "user_attrs": best_trial.user_attrs
    }

    result_dict = {
        "mean_metrics": mean_df,
        "std_metrics": std_df,
        "optimal_configuration": optimal_configuration
    }
    return result_dict

### Run Deep Search for Classification

### Weighted

In [14]:
results_comb_class = run_deep_search_comb_class(comb_class_dict, use_sample_weights=True, model_name="comb_class_deep", n_trials=75)
save_results_pickle(results_comb_class, "comb_class_deep")
results_comb_class["mean_metrics"]


Results saved to search/comb_class_deep_deep_search_results.pkl


,model,type,accuracy,sensitivity,specificity,config
0,comb_class_deep,weighted,0.763958,0.065828,0.982418,"{'lr': 0.00356660993595182, 'dropout_prob': 0...."
1,comb_class_deep,weighted,0.748869,0.143010,0.938462,"{'lr': 0.0005955331235282746, 'dropout_prob': ..."
2,comb_class_deep,weighted,0.762279,0.004706,0.999267,"{'lr': 1.7974344160643104e-05, 'dropout_prob':..."
3,comb_class_deep,weighted,0.762279,0.016471,0.995604,"{'lr': 0.0006468836215398885, 'dropout_prob': ..."
4,comb_class_deep,weighted,0.759484,0.032722,0.986813,"{'lr': 0.0013520870967612822, 'dropout_prob': ..."
...,...,...,...,...,...,...
70,comb_class_deep,weighted,0.761720,0.000000,1.000000,"{'lr': 7.289082720609982e-05, 'dropout_prob': ..."
71,comb_class_deep,weighted,0.765629,0.018769,0.999267,"{'lr': 0.00010213087804239786, 'dropout_prob':..."
72,comb_class_deep,weighted,0.766183,0.037483,0.994139,"{'lr': 0.00014350779165287232, 'dropout_prob':..."
73,comb_class_deep,weighted,0.766186,0.025773,0.997802,"{'lr': 0.00012117497511970936, 'dropout_prob':..."


In [ ]:
results_fitbit_class = run_deep_search_fitbit_class(fitbit_class_dict, use_sample_weights=True, model_name="fitbit_class_deep", n_trials=75)
save_results_pickle(results_fitbit_class, "fitbit_class_deep")
results_fitbit_class["mean_metrics"]


### Unweighted

In [41]:
results_comb_class = run_deep_search_comb_class(comb_class_dict, use_sample_weights=False, model_name="comb_class_deep", n_trials=75)
save_results_pickle(results_comb_class, "comb_class_deep_nw")
results_comb_class["mean_metrics"]

Results saved to search/comb_class_deep_nw_deep_search_results.pkl


,model,type,accuracy,sensitivity,specificity,config
0,comb_class_deep,weighted,0.763390,0.065609,0.981685,"{'batch_size': 48, 'n_filters_0': 8}"
1,comb_class_deep,weighted,0.709254,0.182763,0.873993,"{'batch_size': 32, 'n_filters_0': 64}"
2,comb_class_deep,weighted,0.734371,0.168618,0.911355,"{'batch_size': 48, 'n_filters_0': 32}"
3,comb_class_deep,weighted,0.753904,0.112449,0.954579,"{'batch_size': 64, 'n_filters_0': 16}"
4,comb_class_deep,weighted,0.746647,0.163776,0.928938,"{'batch_size': 64, 'n_filters_0': 24}"
...,...,...,...,...,...,...
70,comb_class_deep,weighted,0.734375,0.182408,0.906960,"{'batch_size': 48, 'n_filters_0': 24}"
71,comb_class_deep,weighted,0.763390,0.065609,0.981685,"{'batch_size': 48, 'n_filters_0': 8}"
72,comb_class_deep,weighted,0.763390,0.065609,0.981685,"{'batch_size': 48, 'n_filters_0': 8}"
73,comb_class_deep,weighted,0.763390,0.065609,0.981685,"{'batch_size': 48, 'n_filters_0': 8}"


In [42]:
results_fitbit_class = run_deep_search_fitbit_class(fitbit_class_dict, use_sample_weights=False, model_name="fitbit_class_deep", n_trials=75)
save_results_pickle(results_fitbit_class, "fitbit_class_deep_nw")
results_fitbit_class["mean_metrics"]


Results saved to search/fitbit_class_deep_nw_deep_search_results.pkl


,model,type,accuracy,sensitivity,specificity,config
0,fitbit_class_deep,weighted,0.701416,0.185062,0.863004,"{'num_epochs': 10, 'lr': 0.0024939966767557526}"
1,fitbit_class_deep,weighted,0.761162,0.002353,0.998535,"{'num_epochs': 5, 'lr': 1.5009952197841239e-05}"
2,fitbit_class_deep,weighted,0.730477,0.112230,0.923810,"{'num_epochs': 6, 'lr': 0.0033768338913100727}"
3,fitbit_class_deep,weighted,0.762277,0.002326,1.000000,"{'num_epochs': 6, 'lr': 0.00014935673948754337}"
4,fitbit_class_deep,weighted,0.738270,0.096033,0.939194,"{'num_epochs': 5, 'lr': 0.004765277633376154}"
...,...,...,...,...,...,...
70,fitbit_class_deep,weighted,0.763955,0.023502,0.995604,"{'num_epochs': 9, 'lr': 0.00023162552300880473}"
71,fitbit_class_deep,weighted,0.761163,0.060985,0.980220,"{'num_epochs': 9, 'lr': 0.00045748783190070806}"
72,fitbit_class_deep,weighted,0.763955,0.030534,0.993407,"{'num_epochs': 9, 'lr': 0.0003002825055670167}"
73,fitbit_class_deep,weighted,0.761165,0.007031,0.997070,"{'num_epochs': 8, 'lr': 0.00019952281945488572}"
